In [1]:
import re

In [2]:
from requests import get
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    
    
    

    
    
    

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">
            <style>
                body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
           


In [3]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
html_soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<style>
                body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
            </style>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb Top 250 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>


In [5]:

movie_containers = html_soup.find_all('tr')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
251


In [6]:
movie_containers

[<tr>
 <th></th>
 <th>Rank &amp; Title</th>
 <th>IMDb Rating</th>
 <th>Your Rating</th>
 <th></th>
 </tr>, <tr>
 <td class="posterColumn">
 <span data-value="1" name="rk"></span>
 <span data-value="9.222336613095289" name="ir"></span>
 <span data-value="7.791552E11" name="us"></span>
 <span data-value="2248724" name="nv"></span>
 <span data-value="-1.7776633869047114" name="ur"></span>
 <a href="/title/tt0111161/"> <img alt="The Shawshank Redemption" height="67" src="https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45"/>
 </a> </td>
 <td class="titleColumn">
       1.
       <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">The Shawshank Redemption</a>
 <span class="secondaryInfo">(1994)</span>
 </td>
 <td class="ratingColumn imdbRating">
 <strong title="9.2 based on 2,248,724 user ratings">9.2</strong>
 </td>
 <td class="ratingColumn">
 <div cla

In [7]:
len(movie_containers)

251

In [8]:
# Lists to store the scraped data in
serialNumbers= []
names = []
years = []
imdb_ratings = []

# Extract data from individual movie container
for container in movie_containers:
# Title column
    try:
        serialNumber= container.find('td', class_ = 'titleColumn').text.split('\n')[1].split()[0]
        serialNumbers.append(serialNumber)
    except:
        serialNumber = 'No Value'
# The name
    try:
        name = container.find('td', class_ = 'titleColumn').find('a').text 
        names.append(name)
    except:
        name= 'No Name'
# The year
    try:
        year = container.find('td', class_ = 'titleColumn').find('span').text
        years.append(year)
    except:
        year= 'No Name'
# The IMDB rating
    try:
        imdb = container.find('td', class_ = 'ratingColumn imdbRating').find('strong').text
        imdb_ratings.append(imdb)
    except:
        imdb = 'No Name'


In [75]:
import pandas as pd
df = pd.DataFrame({'Serial Number': serialNumbers,'Movie': names,
'Year': years,
'imdb Rating': imdb_ratings
})
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Serial Number  250 non-null    object
 1   Movie          250 non-null    object
 2   Year           250 non-null    object
 3   imdb Rating    250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB
None


,Serial Number,Movie,Year,imdb Rating
0,1.,The Shawshank Redemption,(1994),9.2
1,2.,The Godfather,(1972),9.1
2,3.,The Godfather: Part II,(1974),9.0
3,4.,The Dark Knight,(2008),9.0
4,5.,12 Angry Men,(1957),8.9
...,...,...,...,...
245,246.,Munna Bhai M.B.B.S.,(2003),8.0
246,247.,Trois couleurs: Rouge,(1994),8.0
247,248.,Butch Cassidy and the Sundance Kid,(1969),8.0
248,249.,Aladdin,(1992),8.0


In [76]:
df.to_csv('ImDBTopMovies.csv')

In [85]:
import sweetviz
movieList= pd.read_csv('ImDBTopMovies.csv')

In [86]:
movieList.head()

,Unnamed: 0,Serial Number,Movie,Year,imdb Rating
0,0,1.0,The Shawshank Redemption,(1994),9.2
1,1,2.0,The Godfather,(1972),9.1
2,2,3.0,The Godfather: Part II,(1974),9.0
3,3,4.0,The Dark Knight,(2008),9.0
4,4,5.0,12 Angry Men,(1957),8.9


In [87]:
movieList.set_index('Serial Number',drop=True, inplace=True)

In [88]:
movieList.head()

,Unnamed: 0,Movie,Year,imdb Rating
Serial Number,,,,
1.0,0,The Shawshank Redemption,(1994),9.2
2.0,1,The Godfather,(1972),9.1
3.0,2,The Godfather: Part II,(1974),9.0
4.0,3,The Dark Knight,(2008),9.0
5.0,4,12 Angry Men,(1957),8.9


In [89]:
movieList.drop(columns= 'Unnamed: 0', inplace = True)

In [90]:
movieList.head()

,Movie,Year,imdb Rating
Serial Number,,,
1.0,The Shawshank Redemption,(1994),9.2
2.0,The Godfather,(1972),9.1
3.0,The Godfather: Part II,(1974),9.0
4.0,The Dark Knight,(2008),9.0
5.0,12 Angry Men,(1957),8.9


In [91]:
report= sweetviz.analyze([movieList, "Movies"],target_feat='imdb Rating')

:FEATURES DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!


In [92]:
report.show_html('Report.html')